# Analyse des Performances des Pilotes de Formule 1 — Saison 2024

## 1. Introduction & Objectifs
Ce projet vise à **explorer et analyser les performances des pilotes de Formule 1** en utilisant des requêtes SQL exécutées sur une base de données **SQLite**.
L’objectif est de démontrer une approche rigoureuse de **Data Engineering et d’analyse de données sportives**.

Les données proviennent d’un dataset F1 Kaggle 2024, contenant des informations sur les pilotes, les écuries, les courses et les performances individuelles.


## 2. Préparation de l'environnement et chargement des données
Nous commençons par importer les bibliothèques, vérifier les fichiers de données et charger les CSV en DataFrames pandas. Ensuite, nous préparons les formats de temps et créons la base SQLite.

In [ ]:
# Import required libraries and setup database connection
import os, pandas as pd, numpy as np, sqlite3
from sqlalchemy import create_engine
import matplotlib.pyplot as plt, seaborn as sns

DATA_DIR = 'data'
DB_PATH = 'f1_2024.db'
os.makedirs(DATA_DIR, exist_ok=True)

files = {
    'drivers': 'drivers.csv',
    'races': 'races.csv',
    'lap_times': 'lap_times.csv',
    'results': 'results.csv',
    'pit_stops': 'pit_stops.csv'
}

# Check missing files
for k, fn in files.items():
    path = os.path.join(DATA_DIR, fn)
    if not os.path.exists(path):
        print(f"Missing file: {fn}")

# Load data
drivers = pd.read_csv(os.path.join(DATA_DIR, files['drivers']))
races = pd.read_csv(os.path.join(DATA_DIR, files['races']))
lap_times = pd.read_csv(os.path.join(DATA_DIR, files['lap_times']))
results = pd.read_csv(os.path.join(DATA_DIR, files['results']))
pit_stops = pd.read_csv(os.path.join(DATA_DIR, files['pit_stops']))

# Convert time formats to seconds
def time_to_seconds(t):
    if pd.isna(t): return np.nan
    t = str(t).strip()
    if ':' in t:
        m, s = t.split(':'); return float(m)*60+float(s)
    return float(t)

lap_times['lap_time_s'] = lap_times['milliseconds']/1000 if 'milliseconds' in lap_times.columns else lap_times['time'].apply(time_to_seconds)
races_2024 = races[races['year']==2024]
lap_times = lap_times.merge(races_2024[['raceId','year']], on='raceId', how='inner')
lap_times = lap_times[(lap_times['lap_time_s']>10)&(lap_times['lap_time_s']<400)]

# Create SQLite database and store tables
engine = create_engine(f'sqlite:///{DB_PATH}')
drivers.to_sql('drivers', engine, if_exists='replace', index=False)
races_2024.to_sql('races', engine, if_exists='replace', index=False)
lap_times.to_sql('lap_times', engine, if_exists='replace', index=False)
results.to_sql('results', engine, if_exists='replace', index=False)
pit_stops.to_sql('pit_stops', engine, if_exists='replace', index=False)


## 3. Sélection et classement des pilotes
Nous identifions les pilotes les plus performants en fonction des points et calculons les statistiques individuelles sur les temps au tour.

In [ ]:
# Retrieve top 10 drivers based on total points
query = '''
SELECT d.driverId, d.forename || ' ' || d.surname AS driver_name, SUM(r.points) AS total_points
FROM results r
JOIN drivers d ON r.driverId = d.driverId
JOIN races ra ON r.raceId = ra.raceId
WHERE ra.year = 2024
GROUP BY d.driverId
ORDER BY total_points DESC
LIMIT 10
'''
top10 = pd.read_sql(query, engine)

# Merge lap times with driver information
lap_df = pd.read_sql('''SELECT lt.*, d.forename, d.surname FROM lap_times lt JOIN drivers d ON lt.driverId=d.driverId''', engine)
lap_df['driver_name'] = lap_df['forename'] + ' ' + lap_df['surname']

# Compute mean and std lap times per driver
dr_stats = lap_df.groupby('driver_name').agg(n_laps=('lap_time_s','count'), mean_lap_s=('lap_time_s','mean'), std_lap_s=('lap_time_s','std')).reset_index()
drivers_of_interest = top10['driver_name'].tolist()
top10_stats = dr_stats[dr_stats['driver_name'].isin(drivers_of_interest)]


## 4. Comparaison des performances des pilotes
Nous comparons Verstappen et Hamilton avec trois autres pilotes réguliers pour visualiser les différences de performance.

In [ ]:
# Select Verstappen, Hamilton + 3 most consistent drivers
top10_stats = top10_stats.sort_values('std_lap_s')
pilots = ['Max Verstappen', 'Lewis Hamilton']
for n in top10_stats['driver_name']:
    if n not in pilots and len(pilots)<5: pilots.append(n)

selected = dr_stats[dr_stats['driver_name'].isin(pilots)]

# Plot average lap time comparison
plt.figure(figsize=(10,6))
sns.barplot(x='driver_name', y='mean_lap_s', data=selected)
plt.title('Temps moyen au tour - F1 2024')
plt.xticks(rotation=25)
plt.show()


## 5. Statistiques avancées — efficacité et régularité
Nous calculons le temps moyen au tour, la durée moyenne des arrêts aux stands et le taux de victoire pour chaque pilote. Les graphiques permettent une comparaison directe entre les pilotes sélectionnés.

In [ ]:
# 1. Average lap time per driver
mean_lap = dr_stats[['driver_name','mean_lap_s']].sort_values('mean_lap_s')

# 2. Average pit stop duration
pit_stops['duration_s'] = pit_stops['duration'].apply(time_to_seconds)
pit_eff = pit_stops.groupby('driverId').agg(avg_pit_s=('duration_s','mean')).reset_index()
pit_eff = pit_eff.merge(drivers[['driverId','forename','surname']], on='driverId')
pit_eff['driver_name'] = pit_eff['forename'] + ' ' + pit_eff['surname']

# 3. Win rate per driver
win_rate = pd.read_sql('''SELECT d.driverId, d.forename||' '||d.surname AS driver_name, SUM(CASE WHEN r.positionOrder=1 THEN 1 ELSE 0 END)*1.0/COUNT(*) AS win_rate FROM results r JOIN drivers d ON r.driverId=d.driverId JOIN races ra ON r.raceId=ra.raceId WHERE ra.year=2024 GROUP BY d.driverId''', engine)

# Visualize key metrics
fig, axs = plt.subplots(1,3, figsize=(18,5))
sns.barplot(ax=axs[0], x='driver_name', y='mean_lap_s', data=mean_lap)
axs[0].set_title('Average Lap Time (s)')
axs[0].tick_params(axis='x', rotation=60)
sns.barplot(ax=axs[1], x='driver_name', y='avg_pit_s', data=pit_eff)
axs[1].set_title('Average Pit Stop Duration (s)')
axs[1].tick_params(axis='x', rotation=60)
sns.barplot(ax=axs[2], x='driver_name', y='win_rate', data=win_rate)
axs[2].set_title('Win Rate per Driver (2024)')
axs[2].tick_params(axis='x', rotation=60)
plt.tight_layout()
plt.show()
